In [1]:
import numpy as np
import torch
import pandas as pd
from base.Layers.TrainablePatternLayerPNN import TrainablePatternLayerPNN
from PNN.TrainablePNN import TrainablePNN
from GRNN.TrainableGRNN import TrainableGRNN
from PNN.pnn import PNN
from GRNN.grnn import GRNN
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer, load_diabetes, fetch_openml, fetch_olivetti_faces
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from ucimlrepo import fetch_ucirepo

In [2]:
def evaluvate_on_kfold(X, y, model):
    
    if 'GRNN' in model_class.__name__:
        score_func = mean_squared_error
    else:
        score_func = accuracy_score
        
    kf = KFold(n_splits=3)

    scores = []
    
    for i, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, y_train = X[train_index], y[train_index]
        X_test, y_test = X[test_index], y[test_index]
        
        model.fit(X_train, y_train)
        preds = [model.predict([x_test]) for x_test in X_test]
        
        scores.append(score_func(y_true=y_test, y_pred=preds))

    return np.mean(scores)

In [3]:
def grid_search_cv(X, y, model_class, sigma_range, tau_range, n_classes=2, losses=None):

    if losses is None:
        losses = [1] * n_classes

    if 'GRNN' in model_class.__name__:
        score_func = mean_squared_error
    else:
        score_func = accuracy_score
                
    best_sigma = None
    best_score = float('inf') if score_func == mean_squared_error else 0

    for sigma in sigma_range:
        try:
            if 'GRNN' in model_class.__name__:
                score_func = mean_squared_error
                model = model_class(kernel="gaussian", sigma=sigma,
                                        regularization=None, tau=None)
            else:
                score_func = accuracy_score
                model = model_class(kernel="gaussian", sigma=sigma,
                                        regularization=None, tau=None,
                                        n_classes=n_classes, losses=losses)
            score = evaluvate_on_kfold(X, y, model)
            if ((score_func == accuracy_score and score > best_score) or
                (score_func == mean_squared_error and score < best_score)):
                best_score = score
                best_sigma = sigma
        except ZeroDivisionError:
            continue

    results = []
    results.append({
        "Model": f"{model_class.__name__}",
        "Best Sigma": best_sigma,
        "Best Tau": None,
        "Score": best_score
    })

    print(f"Best sigma found: {best_sigma} (Score: {best_score:.4f})")

    regularizations = ['l1', 'l2']
    
    for reg in regularizations:
        best_tau = None
        best_score = float('inf') if score_func == mean_squared_error else 0

        for tau in tau_range:
            try:
                if 'GRNN' in model_class.__name__:
                    trainable_model_class = TrainableGRNN
                    model = trainable_model_class(kernel="gaussian", sigma=best_sigma,
                                    regularization=reg, tau=tau)
                else:
                    trainable_model_class = TrainablePNN
                    model = trainable_model_class(kernel="gaussian", sigma=best_sigma,
                                    regularization=reg, tau=tau,
                                    n_classes=n_classes, losses=losses)
                
                score = evaluvate_on_kfold(X, y, model)
                if ((score_func == accuracy_score and score > best_score) or
                    (score_func == mean_squared_error and score < best_score)):
                    best_score = score
                    best_tau = tau
            except ZeroDivisionError as e:
                continue

        results.append({
            "Model": f"{model_class.__name__} with {reg.upper()}",
            "Best Sigma": best_sigma,
            "Best Tau": best_tau,
            "Score": best_score
        })

    return results

In [4]:
data = load_breast_cancer()
X = data['data']
y_train = data['target']

scaler = StandardScaler()
X_train = scaler.fit_transform(X)

In [5]:
model_class = PNN
sigma_range = np.round(np.arange(0.1, 5.1, 0.1), 2)
tau_range = np.round(np.arange(0.1, 5.1, 0.1), 2)

evals = grid_search_cv(X_train, y_train, model_class, sigma_range, tau_range, n_classes=2, losses=None)

Best sigma found: 0.8 (Score: 0.9561)


/Users/g.vinogradov/Documents/Projects/regularization-pnn-grnn/base/Layers/TrainablePatternLayerPNN.py:98: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1729646995093/work/torch/csrc/utils/tensor_new.cpp:281.)
  input = torch.tensor(input, dtype=torch.float32)


In [6]:
results_df = pd.DataFrame(evals)
results_df['Impovement %'] = np.round((results_df['Score'] / results_df['Score'].iloc[0]) * 100, 2) - 100
results_df

,Model,Best Sigma,Best Tau,Score,Impovement %
0,PNN,0.8,NaN,0.956094,0.00
1,PNN with L1,0.8,2.6,0.957848,0.18
2,PNN with L2,0.8,4.6,0.963121,0.73


In [7]:
results_df.to_latex(float_format="{:.2f}".format)

'\\begin{tabular}{llrrrr}\n\\toprule\n & Model & Best Sigma & Best Tau & Score & Impovement % \\\\\n\\midrule\n0 & PNN & 0.80 & NaN & 0.96 & 0.00 \\\\\n1 & PNN with L1 & 0.80 & 2.60 & 0.96 & 0.18 \\\\\n2 & PNN with L2 & 0.80 & 4.60 & 0.96 & 0.73 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [8]:
df = pd.read_csv('./data/audit_risk.csv')

df = df.dropna()

df = df.drop(columns=['LOCATION_ID'])

X = df.drop(columns=['Risk']).values
y_train = df['Risk'].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X)

In [9]:
model_class = PNN
sigma_range = np.round(np.arange(0.1, 5.1, 0.1), 2)
tau_range = np.round(np.arange(0.1, 5.1, 0.1), 2)

evals = grid_search_cv(X_train, y_train, model_class, sigma_range, tau_range, n_classes=2, losses=None)

Best sigma found: 0.5 (Score: 0.9548)


In [10]:
results_df = pd.DataFrame(evals)
results_df['Impovement %'] = np.round((results_df['Score'] / results_df['Score'].iloc[0]) * 100, 2) - 100
results_df

,Model,Best Sigma,Best Tau,Score,Impovement %
0,PNN,0.5,NaN,0.954840,0.00
1,PNN with L1,0.5,1.2,0.936802,-1.89
2,PNN with L2,0.5,2.6,0.936797,-1.89


In [11]:
data = load_diabetes()

X = data['data']
y_train = data['target']
scaler = StandardScaler()
X_train = scaler.fit_transform(X)

In [12]:
model_class = GRNN
sigma_range = np.round(np.arange(0.1, 5.1, 0.1), 2)
tau_range = np.round(np.arange(0.1, 5.1, 0.1), 2)

evals = grid_search_cv(X_train, y_train, model_class, sigma_range, tau_range, n_classes=2, losses=None)

Best sigma found: 1.0 (Score: 3219.3482)


In [13]:
results_df = pd.DataFrame(evals)
results_df['Impovement %'] = np.round((results_df['Score'].iloc[0] / results_df['Score']) * 100, 2) - 100
results_df

,Model,Best Sigma,Best Tau,Score,Impovement %
0,GRNN,1.0,NaN,3219.348204,0.00
1,GRNN with L1,1.0,2.8,3113.959811,3.38
2,GRNN with L2,1.0,1.1,3080.503068,4.51


In [16]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls"
df = pd.read_excel(url)

X = df.iloc[:, :-1].values
y_train = df.iloc[:, -1].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X)

In [17]:
model_class = GRNN
sigma_range = np.round(np.arange(0.1, 5.1, 0.1), 2)
tau_range = np.round(np.arange(0.1, 5.1, 0.1), 2)

evals = grid_search_cv(X_train, y_train, model_class, sigma_range, tau_range, n_classes=2, losses=None)

Best sigma found: 0.6 (Score: 142.1832)


In [18]:
results_df = pd.DataFrame(evals)
results_df['Impovement %'] = np.round((results_df['Score'].iloc[0] / results_df['Score']) * 100, 2) - 100
results_df

,Model,Best Sigma,Best Tau,Score,Impovement %
0,GRNN,0.6,NaN,142.183214,0.00
1,GRNN with L1,0.6,0.9,136.046653,4.51
2,GRNN with L2,0.6,1.3,137.250853,3.59


In [19]:
stock_portfolio_performance = fetch_ucirepo(id=390)
  
X = stock_portfolio_performance.data.features.iloc[:, :-6]
y_train = stock_portfolio_performance.data.targets['Excess Return'].apply(lambda x: float(x[:-1])).to_numpy()

scaler = StandardScaler()
X_train = scaler.fit_transform(X)

In [20]:
model_class = GRNN
sigma_range = np.round(np.arange(0.1, 5.1, 0.1), 2)
tau_range = np.round(np.arange(0.1, 5.1, 0.1), 2)

evals = grid_search_cv(X_train, y_train, model_class, sigma_range, tau_range, n_classes=2, losses=None)

Best sigma found: 5.0 (Score: 6.7776)


In [21]:
results_df = pd.DataFrame(evals)
results_df['Impovement %'] = np.round((results_df['Score'].iloc[0] / results_df['Score']) * 100, 2) - 100
results_df

,Model,Best Sigma,Best Tau,Score,Impovement %
0,GRNN,5.0,NaN,6.777578,0.00
1,GRNN with L1,5.0,4.0,5.788518,17.09
2,GRNN with L2,5.0,3.6,6.125985,10.64
